In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [2]:
def NSE(obs, sim):
    
    mask_nan = np.isnan(obs) | np.isnan(sim)
    
    obs = obs[~mask_nan] 
    sim = sim[~mask_nan]
    
    return 1 - np.nansum((obs - sim)**2)/np.nansum((obs - np.nanmean(obs))**2)


def KGE(obs, sim):
    mask_nan = np.isnan(obs) | np.isnan(sim)
    
    obs = obs[~mask_nan] 
    sim = sim[~mask_nan]
    
    r = np.corrcoef(obs, sim)[0,1]
    beta  = np.nanmean(sim) / np.nanmean(obs)
    alfa  = np.nanstd(sim) / np.nanstd(obs)
    kge = 1 - np.sqrt( (r-1)**2 + (beta-1)**2 + (alfa-1)**2 )

    
def Bias(obs, sim):
    mask_nan = np.isnan(obs) | np.isnan(sim)
    
    obs = obs[~mask_nan] 
    sim = sim[~mask_nan]
    
    return 100 * ( np.nansum(sim - obs) / np.nansum(obs) )


def MSE(obs, sim):
    mask_nan = np.isnan(obs) | np.isnan(sim)
    
    obs = obs[~mask_nan] 
    sim = sim[~mask_nan]
    
    return np.nanmean((obs - sim)**2)

In [3]:
def periods_constructor(duration, 
                        year_start=1996, 
                        year_end=2009, 
                        stride=1):
    """
    Construction of individual calibration periods
    Input:
    duration: the required duration in calender years, int
    year_start: the first year considered, int
    year_end: the last year considered, int
    stride: default=1
    Output:
    the list of considered calender years
    """
    
    duration = duration - 1
    
    periods=[]
    
    while year_end - duration >= year_start:
        
        period = [year_end-duration, year_end]
        
        periods.append(period)
        
        year_end = year_end - stride
    
    return periods

In [4]:
selected_basins = ['02055100', '02143000',
                   '12143600', '11381500',
                   '03500000', '14306500'
                  ]

In [5]:
HS = 20 # hidden state size of ANN-based models
LB = 4320 # look-back history

results_cal = OrderedDict()
results_val = OrderedDict()

for basin in selected_basins:
    
    results_cal[basin] = OrderedDict()
    results_val[basin] = OrderedDict()
    
    for model_name in ["GRU", "LSTM", "GR4H"]:

        results_cal[basin][model_name] = OrderedDict()
        results_val[basin][model_name] = OrderedDict()

        for duration in [1, 2, 3, 5, 7, 10, 14]:
            
            periods = periods_constructor(duration)
            
            results_cal[basin][model_name][duration] = OrderedDict()
            results_val[basin][model_name][duration] = OrderedDict()
                
            for fname, function in zip(["MSE", "NSE", "KGE", "Bias"], [MSE, NSE, KGE, Bias]):

                results_cal[basin][model_name][duration][fname] = OrderedDict()
                results_val[basin][model_name][duration][fname] = OrderedDict()
                
                for period in periods:

                    # define the first and the last years
                    year_start, year_end = period
                    
                    if model_name == "GR4H":
                        data_cal = pd.read_pickle(f"../results/experiment_results/{basin}/{model_name}/calibration_{year_start}_{year_end}.pkl")
                        data_val = pd.read_pickle(f"../results/experiment_results/{basin}/{model_name}/validation_{year_start}_{year_end}.pkl")
                    else: #GRU,LSTM
                        data_cal = pd.read_pickle(f"../results/experiment_results/{basin}/{model_name}/calibration_{year_start}_{year_end}_{HS}_{LB}.pkl")
                        data_val = pd.read_pickle(f"../results/experiment_results/{basin}/{model_name}/validation_{year_start}_{year_end}_{HS}_{LB}.pkl")
                    
                    # metrics for the calibration period
                    results_cal[basin][model_name][duration][fname][f'{year_start}_{year_end}'] = function(data_cal["Qobs"].to_numpy(),
                                                                                                           data_cal["Qsim"].to_numpy())
                    # metrics for the validation period
                    results_val[basin][model_name][duration][fname][f'{year_start}_{year_end}'] = function(data_val["Qobs"].to_numpy(),
                                                                                                           data_val["Qsim"].to_numpy())

In [6]:
np.save("../results/summary_calibration.npy", results_cal)
np.save("../results/summary_validation.npy", results_val)